<a class="anchor" id="section4"></a>

## Section 4: User-based CF & Item-based CF Recommenders

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('data/split/train_user_artists.csv')

In [ ]:
df_train.head(20)

,Unnamed: 0,name,userID,artistID,weight
0,0,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,6,Diary of Dreams,397,2,56
3,7,Diary of Dreams,560,2,134
4,8,Diary of Dreams,580,2,803
5,9,Diary of Dreams,935,2,428
6,10,Diary of Dreams,1551,2,868
7,13,Diary of Dreams,1679,2,161
8,14,Diary of Dreams,2066,2,83
9,16,Carpathian Forest,1287,3,330


In [ ]:
user_id = df_train.userID.unique().tolist()
# Make dictionary ('original_userID': userID)
user_id_dict = dict(zip(user_id, np.arange(1, len(user_id) + 1)))

artist_id = df_train.artistID.unique().tolist()
# Make dictionary ('original_artistID': artistID)
artist_id_dict = dict(zip(artist_id, np.arange(1, len(artist_id) + 1)))

In [ ]:
# Reindexed dataframe
remap_id_df_train = df_train.replace({'userID': user_id_dict})
remap_id_df_train = remap_id_df_train.replace({'artistID': artist_id_dict})
remap_id_df_train.head(20)

,Unnamed: 0,name,userID,artistID,weight
0,0,MALICE MIZER,1,1,212
1,1,MALICE MIZER,2,1,483
2,6,Diary of Dreams,3,2,56
3,7,Diary of Dreams,4,2,134
4,8,Diary of Dreams,5,2,803
5,9,Diary of Dreams,6,2,428
6,10,Diary of Dreams,7,2,868
7,13,Diary of Dreams,8,2,161
8,14,Diary of Dreams,9,2,83
9,16,Carpathian Forest,10,3,330


In [ ]:
unique_artist = remap_id_df_train.drop_duplicates(subset ="artistID")
unique_artist = unique_artist[['artistID', 'name']]
unique_artist

,artistID,name
0,1,MALICE MIZER
2,2,Diary of Dreams
9,3,Carpathian Forest
11,4,Moi dix Mois
13,5,Bella Morte
...,...,...
55711,12862,Apollo 440
55712,12863,Diamanda Galás
55713,12864,Coptic Rain
55714,12865,Oz Alchemist


In [ ]:
df_train = remap_id_df_train[['artistID', 'userID', 'weight']]
df_train.head()

,artistID,userID,weight
0,1,1,212
1,1,2,483
2,2,3,56
3,2,4,134
4,2,5,803


In [ ]:
df_train_pivot = df_train.pivot(index="artistID", columns="userID", values="weight").fillna(0)
df_train_pivot.shape  # 12866 artist, 1892 user

(12866, 1892)

In [ ]:
df_train_pivot

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892
artistID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,212.0,483.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,56.0,134.0,803.0,428.0,868.0,161.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,330.0,408.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,653.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def create_pivot_table(pd_df):
    data = pd_df.values
    #print(data.shape)
    rows, row_pos = np.unique(data[:, 0], return_inverse=True)
    cols, col_pos = np.unique(data[:, 1], return_inverse=True)
    pivot_matrix = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_matrix[row_pos, col_pos] = data[:, 2]
    print(pivot_matrix.shape)
    return pivot_matrix

In [ ]:
weights = create_pivot_table(df_train)

(12866, 1892)


<a class="anchor" id="section41"></a>

### Section 4.1: Calculate similarity matrix for user & item

* **User Similarity Matrix (#user, #user)**: what is the similarity score between users
* **Item Similarity Matrix (#artist, #artist)**: what is the similarity score between artists

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(weights.T, metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0      #set the NaN values in the matrix to O
print('Shape of User Similarity Matrix:', user_correlation.shape)
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(weights, metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print('Shape of Item Similarity Matrix:', item_correlation.shape)

Shape of User Similarity Matrix: (1892, 1892)
Shape of Item Similarity Matrix: (12866, 12866)


<a class="anchor" id="section42"></a>

### Section 4.2: Predict based on item-based or user-based methods
* User-based:
$$\hat{r}_{ui} = \mu_u + \frac{ \sum\limits_{v \in N^k_i(u)}
\text{sim}(u, v) \cdot (r_{vi} - \mu_v)} {\sum\limits_{v \in
N^k_i(u)} \text{sim}(u, v)}$$
* Item-based:
$$\hat{r}_{ui} = \mu_i + \frac{ \sum\limits_{j \in N^k_u(i)}
\text{sim}(i, j) \cdot (r_{uj} - \mu_j)} {\sum\limits_{j \in
N^k_u(i)} \text{sim}(i, j)}$$

In [ ]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)   #axis=0: apply vertical(row)/axis=1: apply horizontial(row)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
        #pred = 0 + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        mean_item_rating = ratings.mean(axis=0)
        ratings_diff = (ratings - mean_item_rating[np.newaxis, :])
        #pred = 0 + ratings_diff.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        pred = mean_item_rating[np.newaxis, :] + ratings_diff.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred.clip(min=0)

In [ ]:
user_prediction = predict(weights.T, user_correlation, type='user')
item_prediction = predict(weights.T, item_correlation, type='item')

<a class="anchor" id="section43"></a>

### Section 4.3 Recommend artists based on user preference

In [ ]:
user_pred_df = pd.DataFrame(user_prediction, columns =list(unique_artist.artistID))
item_pred_df = pd.DataFrame(item_prediction, columns = list(unique_artist.artistID))

In [ ]:
#adopted from Web Mining exercise 1 ( movie represents artists here)
def recommend_movies(pred_df, userID, movies, original_ratings, num_recommendations, method_name):
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = pred_df.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userID == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'artistID', right_on = 'artistID').
                     sort_values(['weight'], ascending=False)
                 )
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['artistID'].isin(user_full['artistID'])].
         merge(pd.DataFrame(sorted_user_predictions).rename_axis('artistID').reset_index(), how = 'left',
               left_on = 'artistID',
               right_on = 'artistID').
         rename(columns = {user_row_number: method_name}).
         sort_values(method_name, ascending = False).iloc[:num_recommendations, :]
         )
    recommendations.drop(recommendations[recommendations[method_name] == 0.0].index, inplace=True)

    return user_full, recommendations

In [ ]:
# Result example before mapping back userID/artistID
print('Collaborative Filtering RS (user-based)')
user_full, recommendations = recommend_movies(user_pred_df, 1, unique_artist, df_train, 10,'CF_user_pred_rating')
recommendations

Collaborative Filtering RS (user-based)


,artistID,name,CF_user_pred_rating
62,65,Depeche Mode,404.242577
133,136,Pink Floyd,324.802716
125,128,Radiohead,307.104544
192,197,The Beatles,269.856203
156,160,Muse,222.589527
975,987,Björk,147.337312
993,1006,A Perfect Circle,145.078817
129,132,The Cure,114.975050
172,177,Arctic Monkeys,109.213385
3180,3210,311,107.637584


In [ ]:
user_based_result = recommendations.iloc[0:0]
user_based_result.insert(0, "userID", None)

In [ ]:
# Result example before mapping back userID/artistID
print('Collaborative Filtering RS (item-based)')
user_full, recommendations = recommend_movies(item_pred_df, 2, unique_artist, df_train, 10, 'CF_item_pred_rating')
recommendations

Collaborative Filtering RS (item-based)


,artistID,name,CF_item_pred_rating
247,251,Britney Spears,765.186712
78,81,Lady Gaga,429.800993
250,254,Christina Aguilera,358.402680
439,447,Paramore,301.933480
10661,10690,Dominik Eulberg,295.699984
11232,11261,History,295.055359
11233,11262,DJ Rush,294.422695
10679,10708,Mathew Jonson,294.122484
11234,11263,Roman Flügel,294.106628
11235,11264,Moonbootica,294.079672


In [ ]:
item_based_result = recommendations.iloc[0:0]
item_based_result.insert(0, "userID", None)

In [ ]:
# inverse dictionary for mapping back userID 
inv_user_id_dict = {v: k for k, v in user_id_dict.items()}
inv_artist_id_dict = {v: k for k, v in artist_id_dict.items()}

In [ ]:
# Merge User-based result 
no_recommendation_user = []
for i in np.arange(1, len(df_train_pivot.columns) + 1):
    try:
        _, recommendations = recommend_movies(user_pred_df, i, unique_artist, df_train, 10, 'CF_user_pred_rating')
        recommendations['userID'] = i
        user_based_result = user_based_result.append(recommendations, ignore_index=True)
    except:
        print("User {} doesn't exist recommendation".format(i))
        no_recommendation_user.append(i)

In [ ]:
# Map back userID
user_mapback = user_based_result.replace({'userID': inv_user_id_dict})
user_mapback = user_mapback.replace({'artistID': inv_artist_id_dict})
user_based_result = user_mapback.sort_values(by=['userID','CF_user_pred_rating'], ascending = [True, False])
user_based_result.head(20)

,userID,artistID,name,CF_user_pred_rating
2210,2,72,Depeche Mode,509.547262
2211,2,289,Britney Spears,350.841567
2212,2,292,Christina Aguilera,285.149389
2213,2,227,The Beatles,256.969712
2214,2,157,Michael Jackson,249.421767
2215,2,511,U2,248.816051
2216,2,55,Kylie Minogue,235.349364
2217,2,89,Lady Gaga,218.067034
2218,2,154,Radiohead,177.448425
2219,2,993,Simple Minds,161.907262


In [ ]:
user_based_result.to_csv('data/result/user_based_result.csv', index=False)

In [ ]:
# Merge item-based result
no_recommendation_item = []
for i in np.arange(1, len(df_train_pivot.columns) + 1):
    try:
        _, recommendations = recommend_movies(item_pred_df, i, unique_artist, df_train, 10,'CF_item_pred_rating')
        recommendations['userID'] = i
        item_based_result = item_based_result.append(recommendations, ignore_index=True)
    except:
        print("User {} doesn't exist recommendation".format(i))
        no_recommendation_item.append(i)

In [ ]:
# Map back userID
item_mapback = item_based_result.replace({'userID': inv_user_id_dict})
item_mapback = item_mapback.replace({'artistID': inv_artist_id_dict})
item_based_result = item_mapback.sort_values(by=['userID','CF_item_pred_rating'], ascending = [True, False])
item_based_result.head(20)

,userID,artistID,name,CF_item_pred_rating
2210,2,6776,Book of Love,838.788944
2211,2,2571,Bananarama,835.358070
2212,2,6073,Missing Persons,835.321713
2213,2,6778,Boy George,835.251417
2214,2,6779,Josie Cotton,835.233975
2215,2,2562,Arcadia,796.631172
2216,2,289,Britney Spears,777.112525
2217,2,6777,Olivia Newton-John,686.637645
2218,2,2559,Kajagoogoo,612.990992
2219,2,6645,Meiko Kaji,605.323595


In [ ]:
item_based_result.to_csv('data/result/item_based_result.csv', index=False)